## Plots of the results for the Eukaryote datasets

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statistics as s

#### Concatenate all Eukaryote results files.
Euk_balt_results.csv  
Euk_all_results.csv    Euk_RNA_results.csv

In [12]:
results_files = ['Euk_Balt_results.csv','Euk_all_results.csv',    'Euk_RNA_results.csv']
dfs = [] 
for filename in results_files:
    df1 = pd.read_csv(f'../results/{filename}') 
    dfs.append(df1)
df = pd.concat(dfs)

In [14]:
df.head()

,label,label_tax,pool_label,pool_tax,Baltimore,N,Features,k,AUC,Acc,Spec,Sens,FDR,TP,TN,FP,FN
0,Metazoa,kingdom,Eukaryota,superkingdom,dsDNA,120,DNA,1,0.487,0.43,0.81,0.00,1.00,0,13,3,14
1,Metazoa,kingdom,Eukaryota,superkingdom,dsDNA,120,DNA,2,0.946,0.87,0.88,0.86,0.14,12,14,2,2
2,Metazoa,kingdom,Eukaryota,superkingdom,dsDNA,120,DNA,3,0.982,0.90,0.88,0.93,0.13,13,14,2,1
3,Metazoa,kingdom,Eukaryota,superkingdom,dsDNA,120,DNA,4,0.969,0.90,0.88,0.93,0.13,13,14,2,1
4,Metazoa,kingdom,Eukaryota,superkingdom,dsDNA,120,DNA,5,0.964,0.90,0.88,0.93,0.13,13,14,2,1


In [13]:
df.rename(columns = {'label_tax':'tax' }, inplace=True)
df.drop(columns= [ 'pool_label', 'pool tax group'], inplace =True)

con = lambda row :f'{row["label"]}_{row["Baltimore"]}' 
df['Dataset'] = df.apply ( con, axis =1)
fs = lambda row: f'{row["Features"]}_{row["k"]}'
df['Feature set'] = df.apply(fs,axis = 1)

tax_order =['kingdom','phylum','class','order','family','genus','species']
df['tax'] = df['tax'].astype('category')
df['tax'].cat.set_categories(tax_order, inplace=True)
fs_order =['DNA_1','DNA_2','DNA_3','DNA_4','DNA_5','DNA_6','DNA_7','DNA_8','DNA_9','AA_1','AA_2','AA_3','AA_4','PC_1','PC_2','PC_3','PC_4','PC_5','PC_6','Domains_0']
df['Feature set'] = df['Feature set'].astype('category')
df['Feature set'].cat.set_categories(fs_order, inplace=True)
balt_order = ['all','RNA' ,'(+)ssRNA', '(-)ssRNA',  'dsRNA', 'dsDNA','ssDNA']
df['Baltimore'] = df['Baltimore'].astype('category')
df['Baltimore'].cat.set_categories(balt_order, inplace=True)
           

KeyError: "['pool label' 'pool tax group'] not found in axis"

In [ ]:
df.head()

In [ ]:
tax_ss = ['phylum','class','order']
subset = df[(df['tax'].isin(tax_ss))]


sns.set(font_scale=2.5 )
g = sns.catplot(data=subset , col= "tax",col_wrap=1, height=8,aspect =2, 
                  margin_titles=True, col_order=tax_ss, legend_out = True,
                    x = 'Baltimore',y = 'AUC', hue = 'Feature set',
                    kind = 'box',
                    hue_order = fs_order,
                    palette='viridis',
                    order= balt_order,
                    dodge = True,
                    sharex = False,
                     width =.95 ,linewidth =1.5)
                  
g.set_titles('{col_name}' ,fontsize= 80)
g.set(ylim = (0.5,1.02) ) 
plt.subplots_adjust(hspace = 0.2)

## Heat Map

In [ ]:
concat_ds = lambda row: f'{row["tax"][0]}_{row["Dataset"]} '
df['ds']= df.apply(concat_ds , axis=1)

df1 = df.sort_values(['tax','label'])
unique = []
# [unique.append(ds) for ds in list(df1['Dataset']) if ds not in unique ]
for d in (list(df1['ds'])):
    if d not in unique:
        unique.append(d)

dfp = df1.pivot_table(index='ds',columns='Feature set', values ='AUC').reindex(unique)


In [ ]:
dfp.head()

In [ ]:
sns.set(font_scale=1.5)
f, ax = plt.subplots(figsize=(20, 80))
sns.heatmap(dfp,cmap = 'viridis', vmin = 0.5,   ax=ax)

In [ ]:
results_files = ['Euk_balt_results.csv','Euk_all_results.csv',    'Euk_RNA_results.csv']
dfs = [] 
for filename in results_files:
    df1 = pd.read_csv(f'../results/{filename}') 
    dfs.append(df1)
df = pd.concat(dfs)

In [ ]:
df.rename(columns = {'label tax group':'tax', 'positive label':'label','pool label': 'negative label' ,'pool tax group':'negative tax'}, inplace=True)

con = lambda row :f'{row["label"]}_{row["Baltimore"]}' 
df['Dataset'] = df.apply ( con, axis =1)
fs = lambda row: f'{row["Features"]}_{row["k"]}'
df['Feature set'] = df.apply(fs,axis = 1)

tax_order =['kingdom','phylum','class','order','family','genus','species']
df['tax'] = df['tax'].astype('category')
df['tax'].cat.set_categories(tax_order, inplace=True)
fs_order =['DNA_1','DNA_2','DNA_3','DNA_4','DNA_5','DNA_6','DNA_7','DNA_8','DNA_9','AA_1','AA_2','AA_3','AA_4','PC_1','PC_2','PC_3','PC_4','PC_5','PC_6','Domains_0']
df['Feature set'] = df['Feature set'].astype('category')
df['Feature set'].cat.set_categories(fs_order, inplace=True)
balt_order = ['all','RNA' ,'(+)ssRNA', '(-)ssRNA',  'dsRNA', 'dsDNA','ssDNA']
df['Baltimore'] = df['Baltimore'].astype('category')
df['Baltimore'].cat.set_categories(balt_order, inplace=True)
df.drop (columns= ['label','Features','k'], inplace = True)           

In [ ]:
df1 = df.set_index(['Dataset','tax', 'negative label', 'negative tax', 'Baltimore', 'N in class'])

In [ ]:
#df1.reset_index(drop = False,inplace=True)
df1.index.unique()

In [ ]:
dfout =df.pivot_table(index = ['Dataset','tax', 'negative label', 'negative tax', 'Baltimore', 'N in class'],values = 'AUC', columns = 'Feature set', aggfunc = 'mean').reset_index()

In [ ]:
dfout.tail()
